# Match Names 
Rishi Gandhi J022

In [1]:
import pandas as pd

base_names = pd.read_csv('base_names.csv')
name_variations = pd.read_csv('name_variations.csv')


In [9]:
print(base_names.head())
print(name_variations.head())

   Base_Name_ID         Base_Name
0             1        John Smith
1             2    Jennifer Brown
2             3  Michael O'Connor
3             4      Maria Garcia
4             5        Robert Lee
      Variation Matches_With_Base_Name
0  Thomas  King            Thomas King
1    ThomasKing            Thomas King
2  Maria Garcia           Maria Garcia
3     MaryLewis             Mary Lewis
4      Nancy W.           Nancy Wright


# 1. Preprocess Names
Normalize the names in both datasets.

In [11]:
import re

def preprocess_name(name):
    name = name.lower()
    name = ' '.join(name.split())
    if ',' in name:
        parts = name.split(',')
        name = f'{parts[1].strip()} {parts[0].strip()}'
    return name

base_names['processed'] = base_names['Base_Name'].apply(preprocess_name)
name_variations['processed'] = name_variations['Variation'].apply(preprocess_name)


# 2. Fuzzy Matching
Use fuzzy search to find the closest match for each name variation.

In [13]:
from fuzzywuzzy import process

base_name_list = base_names['processed'].tolist()
query_name_list = name_variations['processed'].tolist()

def match_names(query_names, base_names):
    matches = []
    for name in query_names:
        match = process.extractOne(name, base_names)
        matches.append(match)
    return matches

matches = match_names(query_name_list, base_name_list)


C:\Users\rishi\AppData\Roaming\Python\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# 3. Compile Results
Format the matches into a DataFrame and filter based on a similarity threshold.

In [15]:
matches_df = pd.DataFrame(matches, columns=['Matched_Name', 'Score'])
matches_df['Original_Variation'] = name_variations['Variation']
matches_df = matches_df[['Original_Variation', 'Matched_Name', 'Score']]

threshold = 80
filtered_matches = matches_df[matches_df['Score'] >= threshold]

print(filtered_matches)


   Original_Variation    Matched_Name  Score
0        Thomas  King     thomas king    100
1          ThomasKing     thomas king     95
2        Maria Garcia    maria garcia    100
3           MaryLewis      mary lewis     95
4            Nancy W.    nancy wright     90
..                ...             ...    ...
95    Jennifer- Brown  jennifer brown     97
96      Daniel- Scott    daniel scott     96
97           David M.  david martinez     90
98        Paul Allen.      paul allen    100
99        Paul  Allen      paul allen    100

[98 rows x 3 columns]
